#### Backward generation(BWD)

1. generate data after integral

In [1]:
import time

import numpy as np

from backward import generate_bwd

### Testing ...

In [2]:
!wc -l ./data/bwd_sample5000.txt

    3951 ./data/bwd_sample5000.txt


In [15]:
from multiprocessing.pool import ThreadPool

_FINISH = False

sequences_per_process = 10
process_runs = 4

print('generating {} samples in total'.format(
    sequences_per_process * process_runs)
)

start = time.time()
with ThreadPool(processes=14) as p:
    out = []
    r = p.map_async(generate_bwd, [sequences_per_process]*process_runs, callback=out.append)
    r.wait()
    print('done')
    _FINISH = True
    p.terminate()
    p.join()

length = time.time()-start
print("Finished: ", length)  

seconds_per_seq = length / (sequences_per_process * process_runs)
print(seconds_per_seq * 1000000 / 3600)

generating 40 samples in total
process finished
process finished
process finished
process finished
done
Finished:  15.695877075195312
108.99914635552301


In [14]:
len(results[0][0])

10

In [16]:
#from multiprocessing.pool import ThreadPool

_FINISH = False

sequences_per_process = 100
process_runs = 8
round_num = 5

print('generating {} samples in total'.format(
    sequences_per_process * process_runs * round_num)
)

start = time.time()
for i in range(round_num):
    from multiprocessing.pool import ThreadPool
    _FINISH = False
    start = time.time()
    with ThreadPool(processes=8) as p:
        out = []
        r = p.map_async(generate_bwd, [sequences_per_process]*process_runs, callback=out.append)
        r.wait()
        time.sleep(2)
        print('round {} done'.format(i))
        _FINISH = True
        p.terminate()
        p.join()
        time.sleep(10)
    filename = "./data_generation/bwd" + str(sequences_per_process * process_runs) + "_" + str(i+53) + ".txt"
    file = open(filename, "w+")
    for j in range(len(out)):
        sequence =out[0][j]
        file.writelines(sequence)    
    end = time.time()
    print("Finished: {} round in {} seconds".format(i, end-start))

length = time.time()-start
print("Finished: in {} hours".format(length/3600))  

generating 4000 samples in total


Process Process-3693:
Traceback (most recent call last):
  File "/Users/wenjuanyang/opt/anaconda3/envs/symbolic_math/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/wenjuanyang/opt/anaconda3/envs/symbolic_math/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/wenjuanyang/symbolic_math/backward.py", line 46, in f
    d['expr'] = parse_expr(string, local_dict={'x': x}, evaluate=False)
  File "/Users/wenjuanyang/opt/anaconda3/envs/symbolic_math/lib/python3.6/site-packages/sympy/parsing/sympy_parser.py", line 1008, in parse_expr
    return eval_expr(code, local_dict, global_dict)
  File "/Users/wenjuanyang/opt/anaconda3/envs/symbolic_math/lib/python3.6/site-packages/sympy/parsing/sympy_parser.py", line 903, in eval_expr
    code, global_dict, local_dict)  # take local objects in preference
  File "<string>", line 1, in <module>
  File "/Users/wenjuanyang/opt/anaconda3/envs

process finished
process finished
process finished

process finished
process finished
process finished
process finished
process finished


KeyboardInterrupt: 

<pre>
processes=14
sequences_per_process   process_runs         hours for 1 million 
- 16                       4                 94.28
- 4                        16                54.06
- 128                      4                 80.07
- 32                       16                58.18
- 64                       8                 54.56
</pre>

In [ ]:
file = open(r"./data/bwd_sample2048.txt", "w+")
for i in range(len(out)):
    sequence =out[i]
    file.writelines(sequence) 

In [ ]:
filenames = []
for i in range(1):
    filename = "./data/bwd_sample1000_"  +str(i) + ".txt"
    filenames.append(filename)
with open(r"./data/bwd_sample5000.txt", 'w+') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())

In [ ]:
!wc -l ./data/bwd_sample5000.txt